Реализовать подсчёт следующих функций (а также их градиентов и гессианов):

(a) $f(x) = x^2$  (class f1) (1+1+1 баллов)
        
(b)         $f(x) = \sin(3 \cdot \sqrt{x^3} + 2) + x^2$  (class f2) 
        
(c)         $f(x, y) = \frac{(x - 3.3)^2}{4} + \frac{(y + 1.7)^2}{15}$  (class f3) 
        
(d)         $f(\vec x) = \| \vec x \|^2$  (class SquaredL2Norm) 
        
(e)         $f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2$  (class Himmelblau) 
        
(f)         $f(\vec x) = \sum\limits_{i=1}^{n-1}\left[ 100 \cdot \left(x_{i+1} - x_{i}^2 \right)^2 + (1 - x_{i})^2 \right] $  (class Rosenbrok) 

Реализовать функцию minimize - функция для реализации градиентной оптимизации(градиентный спуск +  метод Ньютона).

In [2]:
from math import *
import numpy as np
class f1 :
    def __call__ ( self , x : float ) :
        return x**2
    def grad ( self , x : float ) :
        return 2*x
    def hess ( self , x : float ) :
        return 2
class f2 :
    def __call__ ( self , x : float ) :
        return sin(3*sqrt(x**3)+2)+x**2
    def grad ( self , x : float ) :
        return cos(3*sqrt(x**3)+2)*9/2*(x**(1/2))+2*x
    def hess ( self , x : float ) :
        return -sin(3*sqrt(x**3)+2)*(9/2*x**(1/2))**2+9/4/(x**(1/2))*cos(3*sqrt(x**3)+2)+2
class f3:
    def __call__(self, x: np.ndarray):
        return (x[0]-3.3)**2/4+(x[1]+1.7)**2/15
    def grad(self, x: np.ndarray):
        a=np.ndarray(2, )
        a[0]=(x[0]-3.3)/2
        a[1]=2*(x[1]+1.7)/15
        return a
    def hess(self, x: np.ndarray):
        a=np.ndarray(shape=(2,2), )
        a[0][0]=1/2
        a[1][1]=2/15
        a[0][1]=0
        a[1][0]=0
        return a
class SquaredL2Norm:
    def __call__(self, x: np.ndarray):
        sum=0
        for _x in x:
            sum+=_x**2
        return sum
    def grad(self, x: np.ndarray):
        return x*2
    def hess(self, x: np.ndarray):
        n=x.shape[0]
        a=np.ndarray(shape=(n, n))
        for i in range (n) :
            for j in range(0,i+1) :
                if i==j :
                    a[i][j]=2
                else :
                    a[i][j]=0
                    a[j][i]=0
        return a
class Himmelblau:
    def __call__(self, x: np.ndarray):
        return (x[0]**2+x[1]-11)**2+(x[0]+x[1]**2-7)**2
    def grad(self, x: np.ndarray):
        a=np.ndarray(2, )
        a[0]=4*x[0]*(x[0]**2+x[1]-11)+2*(x[0]+x[1]**2-7)
        a[1]=2*(x[0]**2+x[1]-11)+4*x[1]*(x[0]+x[1]**2-7)
        return a
    def hess(self, x: np.ndarray):
        a=np.ndarray(shape=(2,2), )
        a[0][0]=12*x[0]**2+4*(x[1]-11)+2
        a[1][1]=2+4*(x[0]-7)+12*x[1]**2
        a[0][1]=4*(x[0]+x[1])
        a[1][0]=a[0][1]
        return a
class Rosenbrok:
    def __call__(self, x: np.ndarray):
        n=x.shape[0]
        sum=0
        for i in range(0, n-1) :
            sum+=100*(x[i+1]-x[i]**2)**2+(1-x[i])**2
        return sum
    def grad(self, x: np.ndarray):
        n=x.shape[0]
        a=np.ndarray(n, )
        a[0]=200*(x[1]-x[0]**2)*(-2*x[0])-2*(1-x[0])
        a[n-1]=200*(x[n-1]-x[n-2]**2)
        for i in range(1, n-1) :
            a[i]=200*(x[i]-x[i-1]**2)+200*(x[i+1]-x[i]**2)*(-2*x[i])-2*(1-x[i])
        return a
    def hess(self, x: np.ndarray):
        n=x.shape[0]
        a=np.ndarray(shape=(n, n))
        a[0][0]=-400*x[1]+1200*x[0]**2+2
        a[0][1]=-400*x[0]
        for i in range(2, n) :
            a[0][i]=0
        a[n-1][n-1]=200
        a[n-1][n-2]=-400*x[n-2]
        for i in range(0, n-2) :
            a[n-1][i]=0
        for i in range (1, n-1) :
            for j in range (0, i-1) :
                a[i][j]=0
            for j in range (i+2, n) :
                a[i][j]=0
            a[i][i-1]=-400*x[i-1]
            a[i][i]=200-400*x[i+1]+1200*x[i]**2+2
            a[i][i+1]=-400*x[i]
        return a
def minimize(
        func: callable,
        x_init: np.ndarray,
        learning_rate: callable = lambda x: 0.1,
        method: str = 'gd',
        max_iter: int = 10_000,
        stopping_criteria: str = 'function',
        tolerance: float = 1e-2,
) -> tuple:
    if method=='gd' :
        if stopping_criteria=='points' :
            points_history_list=list()
            points_history_list.append(x_init)
            functions_history_list=list()
            functions_history_list.append(func(x_init))
            grad_history_list=list()
            grad_history_list.append(func.grad(x_init))
            for i in range(1, max_iter+1) :
                x_new=x_init-learning_rate(i-1)*func.grad(x_init)
                points_history_list.append(x_new)
                functions_history_list.append(func(x_new))
                grad_history_list.append(func.grad(x_new))
                if np.linalg.norm(x_init-x_new)<tolerance :
                    x_init=x_new
                    break
                x_init=x_new
            x_opt=x_init
        elif stopping_criteria=='function' :
            points_history_list=list()
            points_history_list.append(x_init)
            functions_history_list=list()
            functions_history_list.append(func(x_init))
            grad_history_list=list()
            grad_history_list.append(func.grad(x_init))
            for i in range(1, max_iter+1) :
                x_new=x_init-learning_rate(i-1)*func.grad(x_init)
                points_history_list.append(x_new)
                functions_history_list.append(func(x_new))
                grad_history_list.append(func.grad(x_new))
                if np.linalg.norm(func(x_init)-func(x_new))<tolerance :
                    x_init=x_new
                    break
                x_init=x_new
            x_opt=x_init
        elif stopping_criteria=='gradient' : 
            points_history_list=list()
            points_history_list.append(x_init)
            functions_history_list=list()
            functions_history_list.append(func(x_init))
            grad_history_list=list()
            grad_history_list.append(func.grad(x_init))
            for i in range(1, max_iter+1) :
                x_new=x_init-learning_rate(i-1)*func.grad(x_init)
                points_history_list.append(x_new)
                functions_history_list.append(func(x_new))
                grad_history_list.append(func.grad(x_new))
                if np.linalg.norm(func.grad(x_new))<tolerance :
                    x_init=x_new
                    break
                x_init=x_new
            x_opt=x_init
            
            
    elif method=='newtone' :
        if stopping_criteria=='points' :
            points_history_list=list()
            points_history_list.append(x_init)
            functions_history_list=list()
            functions_history_list.append(func(x_init))
            grad_history_list=list()
            grad_history_list.append(func.grad(x_init))
            for i in range(1, max_iter+1) :
                if isinstance(func, f1) or  isinstance(func, f2) :
                    x_new=x_init-learning_rate(i-1)/func.hess(x_init)*func.grad(x_init)
                else :
                    x_new=x_init-learning_rate(i-1)*np.matmul(np.linalg.inv(func.hess(x_init)), func.grad(x_init))
                points_history_list.append(x_new)
                functions_history_list.append(func(x_new))
                grad_history_list.append(func.grad(x_new))
                if np.linalg.norm(x_init-x_new)<tolerance :
                    x_init=x_new
                    break
                x_init=x_new
            x_opt=x_init
        elif stopping_criteria=='function' :
            points_history_list=list()
            points_history_list.append(x_init)
            functions_history_list=list()
            functions_history_list.append(func(x_init))
            grad_history_list=list()
            grad_history_list.append(func.grad(x_init))
            for i in range(1, max_iter+1) :
                if isinstance(func, f1) or  isinstance(func, f2) :
                    x_new=x_init-learning_rate(i-1)/func.hess(x_init)*func.grad(x_init)
                else :
                    x_new=x_init-learning_rate(i-1)*np.matmul(np.linalg.inv(func.hess(x_init)), func.grad(x_init))
                points_history_list.append(x_new)
                functions_history_list.append(func(x_new))
                grad_history_list.append(func.grad(x_new))
                if np.linalg.norm(func(x_init)-func(x_new))<tolerance :
                    x_init=x_new
                    break
                x_init=x_new
            x_opt=x_init
        elif stopping_criteria=='gradient' : 
            points_history_list=list()
            points_history_list.append(x_init)
            functions_history_list=list()
            functions_history_list.append(func(x_init))
            grad_history_list=list()
            grad_history_list.append(func.grad(x_init))
            for i in range(1, max_iter+1) :
                if isinstance(func, f1) or  isinstance(func, f2) :
                    x_new=x_init-learning_rate(i-1)/func.hess(x_init)*func.grad(x_init)
                else :
                    x_new=x_init-learning_rate(i-1)*np.matmul(np.linalg.inv(func.hess(x_init)), func.grad(x_init))
                points_history_list.append(x_new)
                functions_history_list.append(func(x_new))
                grad_history_list.append(func.grad(x_new))
                if np.linalg.norm(func.grad(x_new))<tolerance :
                    x_init=x_new
                    break
                x_init=x_new
            x_opt=x_init

    return x_opt, points_history_list, functions_history_list, grad_history_list